# 01_PREPROCESSING: World Bank FDI & GDP Data

**Mục đích:** Xử lý dữ liệu từ World Bank CSV files

**Output:** `ASEAN_FDI_GDP_Data_Final.csv` (150 quan sát sạch sẽ)

---

## 1. Import thư viện

In [17]:
import pandas as pd
import numpy as np
import os
import glob
from datetime import datetime

print("✅ Thư viện imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

✅ Thư viện imported successfully!
Pandas version: 2.3.3
NumPy version: 2.3.5


## 2. Cấu hình

In [18]:
# Các quốc gia ASEAN (sử dụng mã ISO Alpha-3) - 9 QUỐC GIA (loại bỏ Myanmar và Timor-Leste)
COUNTRIES = {
    'VNM': 'Viet Nam',
    'IDN': 'Indonesia',
    'THA': 'Thailand',
    'PHL': 'Philippines',
    'MYS': 'Malaysia',
    'SGP': 'Singapore',
    'BRN': 'Brunei Darussalam',
    'KHM': 'Cambodia',
    'LAO': 'Lao PDR'
}

# Các chỉ số
INDICATORS = {
    'BX.KLT.DINV.CD.WD': 'FDI',
    'NY.GDP.MKTP.KD.ZG': 'GDP_Growth',
    'NE.EXP.GNFS.CD.ZS': 'Exports_pct_GDP',
    'NE.IMP.GNFS.CD.ZS': 'Imports_pct_GDP'
}

# Khoảng thời gian
START_YEAR = 2000
END_YEAR = 2024

# Đường dẫn
RAW_DATA_DIR = '../01_Data/01_Raw_CSV/'
PROCESSED_DATA_DIR = '../01_Data/02_Processed/'

# Tạo thư mục nếu chưa tồn tại
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)

print(f"✅ Configuration loaded:")
print(f"   - Countries: {len(COUNTRIES)} ASEAN countries (excluding Myanmar and Timor-Leste)")
print(f"   - Indicators: {len(INDICATORS)}")
print(f"   - Time period: {START_YEAR}-{END_YEAR} ({END_YEAR - START_YEAR + 1} years)")
print(f"   - Expected observations: {len(COUNTRIES) * (END_YEAR - START_YEAR + 1)} = {len(COUNTRIES)} countries × {END_YEAR - START_YEAR + 1} years")

✅ Configuration loaded:
   - Countries: 9 ASEAN countries (excluding Myanmar and Timor-Leste)
   - Indicators: 4
   - Time period: 2000-2024 (25 years)
   - Expected observations: 225 = 9 countries × 25 years


## 3. Hàm xử lý file CSV

In [19]:
def process_world_bank_csv(filepath, indicator_name, countries_dict):
    """
    Xử lý file CSV từ World Bank
    """
    print(f"\n→ Đang xử lý file: {os.path.basename(filepath)}")
    
    try:
        # Đọc file, bỏ 4 hàng header
        df = pd.read_csv(filepath, skiprows=4)
        print(f"  ✓ File loaded: {len(df)} dòng")
        
        # Lấy country codes
        country_codes = list(countries_dict.keys())
        
        # Lọc chỉ các nước ASEAN
        df_filtered = df[df['Country Code'].isin(country_codes)].copy()
        print(f"  ✓ Filtered: {len(df_filtered)} quốc gia ASEAN")
        
        # Xử lý dữ liệu
        results = []
        
        for idx, row in df_filtered.iterrows():
            country_code = row['Country Code']
            country_name = countries_dict[country_code]
            
            # Lặp qua từng năm
            for year in range(START_YEAR, END_YEAR + 1):
                year_str = str(year)
                
                if year_str in df.columns:
                    value = row[year_str]
                    
                    if pd.notna(value):
                        try:
                            value = float(value)
                            results.append({
                                'Country': country_name,
                                'Country_Code': country_code,
                                'Year': year,
                                'Indicator': indicator_name,
                                'Value': value
                            })
                        except (ValueError, TypeError):
                            pass
        
        result_df = pd.DataFrame(results)
        print(f"  ✓ Processed: {len(result_df)} data points")
        
        return result_df
    
    except FileNotFoundError:
        print(f"  ❌ ERROR: File not found: {filepath}")
        return None
    except Exception as e:
        print(f"  ❌ ERROR: {e}")
        return None

print("✅ Hàm process_world_bank_csv được định nghĩa")

✅ Hàm process_world_bank_csv được định nghĩa


## 4. Tìm và xử lý các file CSV

In [20]:
# Tìm các file CSV
csv_files = glob.glob('../01_Data/01_Raw_CSV/*.csv')
print(f"\nTìm thấy {len(csv_files)} file CSV:")
for file in csv_files:
    print(f"  - {os.path.basename(file)}")



Tìm thấy 4 file CSV:
  - API_BX.KLT.DINV.CD.WD_DS2_en_csv_v2_2672.csv
  - API_NE.EXP.GNFS.ZS_DS2_en_csv_v2_4843.csv
  - API_NE.IMP.GNFS.ZS_DS2_en_csv_v2_2642.csv
  - API_NY.GDP.MKTP.KD.ZG_DS2_en_csv_v2_2509.csv


In [21]:
# Kiểm tra tên quốc gia trong file CSV
import pandas as pd

# Đọc một file mẫu để xem tên quốc gia
sample_file = csv_files[0]
print(f"\n→ Kiểm tra file: {os.path.basename(sample_file)}")

# Đọc file (bỏ qua 4 dòng đầu theo cấu trúc World Bank)
df_sample = pd.read_csv(sample_file, skiprows=4)
print(f"\n→ Các cột trong file: {list(df_sample.columns)}")

# Hiển thị các quốc gia trong file
print(f"\n→ Tổng số quốc gia/vùng: {len(df_sample)}")
print("\n→ 20 quốc gia đầu tiên trong file:")
print(df_sample[['Country Name', 'Country Code']].head(20))

# Tìm các quốc gia ASEAN trong file
print("\n→ Kiểm tra các quốc gia ASEAN trong danh sách COUNTRIES:")
for country_name, country_code in COUNTRIES.items():
    matches = df_sample[df_sample['Country Name'] == country_name]
    if len(matches) > 0:
        print(f"  ✓ Tìm thấy: {country_name} ({country_code})")
    else:
        print(f"  ✗ Không tìm thấy: {country_name} ({country_code})")
        # Tìm các tên tương tự
        similar = df_sample[df_sample['Country Name'].str.contains(country_name.split()[0], case=False, na=False)]
        if len(similar) > 0:
            print(f"    → Có thể là: {list(similar['Country Name'].values)}")



→ Kiểm tra file: API_BX.KLT.DINV.CD.WD_DS2_en_csv_v2_2672.csv

→ Các cột trong file: ['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code', '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']

→ Tổng số quốc gia/vùng: 266

→ 20 quốc gia đầu tiên trong file:
                   Country Name Country Code
0                         Aruba          ABW
1   Africa Eastern and Southern          AFE
2                   Afghanistan          AFG
3    Africa Western and Central          AFW
4                        Angola   

In [22]:
# Tìm mã code thực tế của các quốc gia ASEAN
print("\n→ Tìm mã Country Code thực tế cho các quốc gia ASEAN:")
asean_countries_fullname = ['Vietnam', 'Indonesia', 'Thailand', 'Philippines', 'Malaysia', 'Singapore']

for country_fullname in asean_countries_fullname:
    matches = df_sample[df_sample['Country Name'] == country_fullname]
    if len(matches) > 0:
        code = matches.iloc[0]['Country Code']
        print(f"  ✓ {country_fullname:20s} → Code: {code}")
    else:
        print(f"  ✗ {country_fullname:20s} → Không tìm thấy")



→ Tìm mã Country Code thực tế cho các quốc gia ASEAN:
  ✗ Vietnam              → Không tìm thấy
  ✓ Indonesia            → Code: IDN
  ✓ Thailand             → Code: THA
  ✓ Philippines          → Code: PHL
  ✓ Malaysia             → Code: MYS
  ✓ Singapore            → Code: SGP


In [23]:
# Tìm Vietnam với các biến thể tên
print("\n→ Tìm Vietnam trong file:")
vietnam_variants = df_sample[df_sample['Country Name'].str.contains('Viet', case=False, na=False)]
print(vietnam_variants[['Country Name', 'Country Code']])


→ Tìm Vietnam trong file:
    Country Name Country Code
257     Viet Nam          VNM


## 5. Xử lý từng file

In [24]:
print("\n" + "="*90)
print("PHẦN 2: LỌC VÀ HỢP NHẤT DỮ LIỆU")
print("="*90)

all_data = []

# FDI
fdi_files = [f for f in csv_files if 'FDI' in f or 'BX.KLT.DINV' in f]
if fdi_files:
    df_fdi = process_world_bank_csv(fdi_files[0], 'FDI', COUNTRIES)
    if df_fdi is not None:
        all_data.append(df_fdi)
else:
    print("\n⚠️ Không tìm thấy file FDI")

# GDP Growth
gdp_files = [f for f in csv_files if 'GDP' in f or 'NY.GDP.MKTP.KD.ZG' in f]
if gdp_files:
    df_gdp = process_world_bank_csv(gdp_files[0], 'GDP_Growth', COUNTRIES)
    if df_gdp is not None:
        all_data.append(df_gdp)
else:
    print("\n⚠️ Không tìm thấy file GDP Growth")

# Exports
export_files = [f for f in csv_files if 'Export' in f or 'NE.EXP' in f]
if export_files:
    df_export = process_world_bank_csv(export_files[0], 'Exports_pct_GDP', COUNTRIES)
    if df_export is not None:
        all_data.append(df_export)
else:
    print("\n⚠️ Không tìm thấy file Exports")

# Imports
import_files = [f for f in csv_files if 'Import' in f or 'NE.IMP' in f]
if import_files:
    df_import = process_world_bank_csv(import_files[0], 'Imports_pct_GDP', COUNTRIES)
    if df_import is not None:
        all_data.append(df_import)
else:
    print("\n⚠️ Không tìm thấy file Imports")

print(f"\n→ Hợp nhất {len(all_data)} indicator...")
if len(all_data) > 0:
    df_combined = pd.concat(all_data, ignore_index=True)
    print(f"✓ Hợp nhất: {len(df_combined)} records")
else:
    print("❌ Không có dữ liệu để xử lý!")


PHẦN 2: LỌC VÀ HỢP NHẤT DỮ LIỆU

→ Đang xử lý file: API_BX.KLT.DINV.CD.WD_DS2_en_csv_v2_2672.csv
  ✓ File loaded: 266 dòng
  ✓ Filtered: 9 quốc gia ASEAN
  ✓ Processed: 225 data points

→ Đang xử lý file: API_NY.GDP.MKTP.KD.ZG_DS2_en_csv_v2_2509.csv
  ✓ File loaded: 266 dòng
  ✓ Filtered: 9 quốc gia ASEAN
  ✓ Processed: 225 data points

→ Đang xử lý file: API_NE.EXP.GNFS.ZS_DS2_en_csv_v2_4843.csv
  ✓ File loaded: 266 dòng
  ✓ Filtered: 9 quốc gia ASEAN
  ✓ Processed: 216 data points

→ Đang xử lý file: API_NE.IMP.GNFS.ZS_DS2_en_csv_v2_2642.csv
  ✓ File loaded: 266 dòng
  ✓ Filtered: 9 quốc gia ASEAN
  ✓ Processed: 216 data points

→ Hợp nhất 4 indicator...
✓ Hợp nhất: 882 records


## 6. Chuyển đổi dạng Wide Format

In [25]:
print("\n" + "="*90)
print("PHẦN 3: CHUYỂN ĐỔI DẠNG DỮ LIỆU")
print("="*90)

print("\n→ Chuyển đổi từ long format sang wide format...")

df_pivot = df_combined.pivot_table(
    index=['Country', 'Country_Code', 'Year'],
    columns='Indicator',
    values='Value',
    aggfunc='first'
).reset_index()

print(f"✓ Dạng mới: {df_pivot.shape[0]} dòng × {df_pivot.shape[1]} cột")
print(f"✓ Cột: {list(df_pivot.columns)}")


PHẦN 3: CHUYỂN ĐỔI DẠNG DỮ LIỆU

→ Chuyển đổi từ long format sang wide format...
✓ Dạng mới: 225 dòng × 7 cột
✓ Cột: ['Country', 'Country_Code', 'Year', 'Exports_pct_GDP', 'FDI', 'GDP_Growth', 'Imports_pct_GDP']


## 7. Làm sạch dữ liệu

In [26]:
print("\n" + "="*90)
print("PHẦN 4: LÀM SẠCH DỮ LIỆU")
print("="*90)

print("\n→ Kiểm tra Missing Values:")
print(df_pivot.isnull().sum())

print("\n→ Xử lý Missing Values:")

for col in ['FDI', 'GDP_Growth', 'Exports_pct_GDP', 'Imports_pct_GDP']:
    if col in df_pivot.columns:
        missing_count = df_pivot[col].isnull().sum()
        if missing_count > 0:
            print(f"   {col}: {missing_count} missing values")
            df_pivot[col] = df_pivot.groupby('Country')[col].transform(
                lambda x: x.interpolate(method='linear', limit_direction='both')
            )
            print(f"   ✓ Đã xử lý bằng linear interpolation")
        else:
            print(f"   ✓ {col}: Hoàn chỉnh (không missing values)")


PHẦN 4: LÀM SẠCH DỮ LIỆU

→ Kiểm tra Missing Values:
Indicator
Country            0
Country_Code       0
Year               0
Exports_pct_GDP    9
FDI                0
GDP_Growth         0
Imports_pct_GDP    9
dtype: int64

→ Xử lý Missing Values:
   ✓ FDI: Hoàn chỉnh (không missing values)
   ✓ GDP_Growth: Hoàn chỉnh (không missing values)
   Exports_pct_GDP: 9 missing values
   ✓ Đã xử lý bằng linear interpolation
   Imports_pct_GDP: 9 missing values
   ✓ Đã xử lý bằng linear interpolation


## 8. Tính biến phụ

In [27]:
print("\n" + "="*90)
print("PHẦN 5: TẠO BIẾN TÍNH TOÁN")
print("="*90)

print("\n→ Tính Trade Openness (Exports + Imports) / GDP * 100...")

if 'Exports_pct_GDP' in df_pivot.columns and 'Imports_pct_GDP' in df_pivot.columns:
    df_pivot['Trade_Openness'] = df_pivot['Exports_pct_GDP'] + df_pivot['Imports_pct_GDP']
    print(f"   ✓ Trade_Openness = Exports + Imports")
    
    vietnam_2000 = df_pivot[(df_pivot['Country']=='Vietnam') & (df_pivot['Year']==2000)]['Trade_Openness'].values
    if len(vietnam_2000) > 0:
        print(f"   ✓ Ví dụ Vietnam 2000: {vietnam_2000[0]:.2f}%")


PHẦN 5: TẠO BIẾN TÍNH TOÁN

→ Tính Trade Openness (Exports + Imports) / GDP * 100...
   ✓ Trade_Openness = Exports + Imports


## 9. Kiểm tra chất lượng dữ liệu

In [28]:
print("\n" + "="*90)
print("PHẦN 6: KIỂM TRA CHẤT LƯỢNG DỮ LIỆU")
print("="*90)

print("\n→ Thống kê mô tả:")
print(df_pivot[['FDI', 'GDP_Growth', 'Trade_Openness']].describe().round(3))

print("\n→ Dữ liệu theo quốc gia:")
for country in sorted(df_pivot['Country'].unique()):
    df_country = df_pivot[df_pivot['Country'] == country]
    print(f"\n   {country}:")
    print(f"      - Số năm: {len(df_country)}")
    print(f"      - Năm: {df_country['Year'].min()}-{df_country['Year'].max()}")
    print(f"      - FDI: {df_country['FDI'].min():.2f} → {df_country['FDI'].max():.2f} tỷ USD")
    print(f"      - GDP Growth: {df_country['GDP_Growth'].min():.2f}% → {df_country['GDP_Growth'].max():.2f}%")


PHẦN 6: KIỂM TRA CHẤT LƯỢNG DỮ LIỆU

→ Thống kê mô tả:
Indicator           FDI  GDP_Growth  Trade_Openness
count      2.250000e+02     225.000         225.000
mean       1.229850e+10       4.781         134.590
std        2.442036e+10       3.301          88.715
min       -4.550355e+09      -9.518          32.972
25%        7.756420e+08       3.555          75.092
50%        4.376053e+09       5.294         117.253
75%        1.260000e+10       6.899         142.721
max        1.519410e+11      14.520         437.327

→ Dữ liệu theo quốc gia:

   Brunei Darussalam:
      - Số năm: 25
      - Năm: 2000-2024
      - FDI: -290169984.90 → 864905530.00 tỷ USD
      - GDP Growth: -3.90% → 4.20%

   Cambodia:
      - Số năm: 25
      - Năm: 2000-2024
      - FDI: 81580650.56 → 4394647334.00 tỷ USD
      - GDP Growth: -3.56% → 13.30%

   Indonesia:
      - Số năm: 25
      - Năm: 2000-2024
      - FDI: -4550355286.00 → 25120732060.00 tỷ USD
      - GDP Growth: -2.07% → 6.35%

   Lao PDR:
    

## 10. Sắp xếp và định dạng cuối cùng

In [29]:
print("\n" + "="*90)
print("PHẦN 7: SẮP XẾP VÀ ĐỊNH DẠNG CUỐI CÙNG")
print("="*90)

# Sắp xếp theo Country và Year
df_final = df_pivot.sort_values(['Country', 'Year']).reset_index(drop=True)

# Chọn các cột cần thiết
columns_final = ['Country', 'Country_Code', 'Year', 'FDI', 'GDP_Growth', 
                 'Exports_pct_GDP', 'Imports_pct_GDP', 'Trade_Openness']

columns_final = [col for col in columns_final if col in df_final.columns]
df_final = df_final[columns_final]

print(f"\n→ Định dạng cuối cùng:")
print(f"   - Số dòng: {len(df_final)}")
print(f"   - Số cột: {len(df_final.columns)}")
print(f"   - Cột: {list(df_final.columns)}")

print("\n→ 10 dòng đầu:")
print(df_final.head(10))

print("\n→ 10 dòng cuối:")
print(df_final.tail(10))


PHẦN 7: SẮP XẾP VÀ ĐỊNH DẠNG CUỐI CÙNG

→ Định dạng cuối cùng:
   - Số dòng: 225
   - Số cột: 8
   - Cột: ['Country', 'Country_Code', 'Year', 'FDI', 'GDP_Growth', 'Exports_pct_GDP', 'Imports_pct_GDP', 'Trade_Openness']

→ 10 dòng đầu:
Indicator            Country Country_Code  Year           FDI  GDP_Growth  \
0          Brunei Darussalam          BRN  2000  5.496072e+08    3.474676   
1          Brunei Darussalam          BRN  2001  6.069464e+07    1.466061   
2          Brunei Darussalam          BRN  2002  2.296720e+08    3.961439   
3          Brunei Darussalam          BRN  2003  1.238209e+08    3.583429   
4          Brunei Darussalam          BRN  2004  1.132059e+08    0.104538   
5          Brunei Darussalam          BRN  2005  1.750685e+08   -0.003925   
6          Brunei Darussalam          BRN  2006  8.783913e+07    4.098413   
7          Brunei Darussalam          BRN  2007  2.576357e+08   -3.763623   
8          Brunei Darussalam          BRN  2008  2.221845e+08   -3.9007

## 11. Lưu file

In [30]:
print("\n" + "="*90)
print("PHẦN 8: LƯU FILE")
print("="*90)

# Tạo tên file với timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_filename = f"ASEAN_FDI_GDP_Data_Processed_{timestamp}.csv"

# Lưu file
df_final.to_csv(f'../01_Data/02_Processed/{output_filename}', index=False)
print(f"\n✓ File đã lưu: {output_filename}")

# Lưu thêm file không có timestamp
df_final.to_csv('../01_Data/02_Processed/ASEAN_FDI_GDP_Data_Final.csv', index=False)
print(f"✓ File đã lưu: ASEAN_FDI_GDP_Data_Final.csv")


PHẦN 8: LƯU FILE

✓ File đã lưu: ASEAN_FDI_GDP_Data_Processed_20251210_145138.csv
✓ File đã lưu: ASEAN_FDI_GDP_Data_Final.csv

✓ File đã lưu: ASEAN_FDI_GDP_Data_Processed_20251210_145138.csv
✓ File đã lưu: ASEAN_FDI_GDP_Data_Final.csv


## 12. Kiểm chứng dữ liệu

In [31]:
print("\n" + "="*90)
print("PHẦN 9: KIỂM CHỨNG DỮ LIỆU")
print("="*90)

print("\n→ Kiểm tra Vietnam 2000:")
vietnam_2000 = df_final[(df_final['Country'] == 'Vietnam') & (df_final['Year'] == 2000)]
if len(vietnam_2000) > 0:
    print(vietnam_2000.to_string(index=False))
    fdi_value = vietnam_2000['FDI'].values[0]
    print(f"\n   FDI: {fdi_value:.3f} tỷ USD = {fdi_value * 1e9:,.0f} USD ✓")
else:
    print("   ❌ Không tìm thấy dữ liệu Vietnam 2000")


PHẦN 9: KIỂM CHỨNG DỮ LIỆU

→ Kiểm tra Vietnam 2000:
   ❌ Không tìm thấy dữ liệu Vietnam 2000


## 13. Thống kê tương quan

In [32]:
print("\n" + "="*90)
print("PHẦN 10: THỐNG KÊ TƯƠNG QUAN")
print("="*90)

print("\n→ Tương quan với Vietnam:")
df_vietnam = df_final[df_final['Country'] == 'Vietnam']
if len(df_vietnam) > 0:
    corr = df_vietnam[['FDI', 'GDP_Growth', 'Trade_Openness']].corr()
    print(corr.round(3))
else:
    print("   ❌ Không tìm thấy dữ liệu Vietnam")


PHẦN 10: THỐNG KÊ TƯƠNG QUAN

→ Tương quan với Vietnam:
   ❌ Không tìm thấy dữ liệu Vietnam


## 🎉 HOÀN THÀNH!

✅ **Preprocessing xong!**

📄 **File output:**
- `ASEAN_FDI_GDP_Data_Final.csv` - File chính (150 quan sát, 8 cột)

**Bước tiếp theo:**
1. Mở notebook `02_Model_LinearRegression.ipynb` để chạy Linear Regression
2. Hoặc chạy các model khác trong thư mục `02_Scripts/`
